In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from osgeo import gdal
import xarray as xr
import numpy as np
import rasterio
import rioxarray
import pandas as pd
import pickle

## slope factor

In [ ]:
# da = rioxarray.open_rasterio("/content/drive/MyDrive/rsa3.tif", masked=True)
slope_classidied = rioxarray.open_rasterio("/mnt/e/Term2/payan_name/Proposal/extentions/RSA/Reclass_Slop31.tif", masked=True)
# Assuming you have a DataArray named
slope_classidied.name = 'rsa'

# Convert DataArray to Dataset
slope_factor = slope_classidied.to_dataset()
slope_factor


In [ ]:
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 1, 0, slope_factor.rsa.data)
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 2, 0.2, slope_factor.rsa.data)
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 3, 0.4, slope_factor.rsa.data)
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 4, 0.6, slope_factor.rsa.data)
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 5, 0.8, slope_factor.rsa.data)
slope_factor.rsa.data = np.where(slope_factor.rsa.data == 6, 1, slope_factor.rsa.data)


In [ ]:
# df = slope_factor.rsa.to_dataframe()

In [ ]:
# maped_dict = {
#     1 : 0,
#     2 : 0.2,
#     3 : 0.4,
#     4 : 0.6,
#     5 : 0.8,
#     6 : 1
# }


# slope_factor['coefficient'] = slope_factor['rsa'].map(maped_dict)



In [ ]:
# rsa_name_da = df.to_xarray()
# rsa_name_da

# read flow length

In [ ]:
flowlen = rioxarray.open_rasterio("/mnt/e/Term2/payan_name/Proposal/extentions/RSA/flowlen.tif", masked=True)
# flowlen = rioxarray.open_rasterio("/content/drive/MyDrive/flowlen.tif", masked=True)

# Assuming you have a DataArray named
flowlen.name = 'length'

# Convert DataArray to Dataset
fl = flowlen.to_dataset()
fl




In [ ]:
hight = np.nanmax(fl['length'].values)
low = np.nanmin(fl['length'].values)


In [ ]:
# df_flowlen = fl.to_dataframe()
# df_flowlen

In [ ]:
fl.factor.data = np.vectorize(lambda m, l, h: (m - l) / (h - l))(
    m = fl.length,
    l = low,
    h = hight
)




# df_flowlen["factor"] = 1 - ((df_flowlen['length'] - low) / (hight - low))
# df_flowlen

In [ ]:
flowlen_name_da = df_flowlen.to_xarray()
flowlen_name_da

In [ ]:
rsa_name_da["coefficient"][0][3000][3000]

In [ ]:
flowlen_name_da["factor"][0][3000][3000]

In [ ]:
a = rsa_name_da["coefficient"] + flowlen_name_da["factor"]
a_min = np.nanmin(a.values)
a_max = np.nanmax(a.values)

b = (a - a_min) / (a_max - a_min)

b

In [ ]:
# Assuming you have a DataArray named
b.name = 'rsa_final'

# Convert DataArray to Dataset
b_dataset = b.to_dataset()
b_dataset.rsa_final.plot()

In [ ]:
b_dataset

In [ ]:
c = rsa_name_da["coefficient"] * flowlen_name_da["factor"]

c_min = np.nanmin(c.values)
c_max = np.nanmax(c.values)

c.name = 'rsa_final_multiplication'

c_dataset = c.to_dataset()
c_dataset.rsa_final_multiplication.plot()


In [ ]:
d = (rsa_name_da["coefficient"] * flowlen_name_da["factor"])**0.5

d_min = np.nanmin(d.values)
d_max = np.nanmax(d.values)

d.name = 'rsa_final_geometric_mean'

d_dataset = d.to_dataset()
d_dataset.rsa_final_geometric_mean.plot()

In [ ]:
e = (rsa_name_da["coefficient"] + flowlen_name_da["factor"]) / 2

e_min = np.nanmin(e.values)
e_max = np.nanmax(e.values)

e.name = 'rsa_final_arithmetic_mean'

e_dataset = e.to_dataset()
e_dataset.rsa_final_arithmetic_mean.plot()